
# Assignment 3
* Li Sun (ls1229) 
* Yifei Wang (yw485)

## C MORPH VIDEO
### 1 Method Brief Description of the Approach Used:
1. To determine the image correspondence, we implemented an interactive display to ask the user to click 12 correspondence points in sequence. (eg: 2 on eyes, 3 on nose, 3 on mouth, 1 on chin, 1 on top of forehead, 2 beside ears). (Only 8 shown in below picture)  
![](D:\OneDrive\COSC579\HW31.jpg)
2.	 When computing the triangulation, we used Delauney Triangulation (A function in Scipy package) based on the correspondences.  After triangulation, we have triangle correspondences.  
![](D:\OneDrive\COSC579\HW32.jpg)
3. Then we calculate the new triangle based on alpha and two correspond triangle: `newtri = t1points*alpha + t2points*(1-alpha)`
4. Use new triangle and old triangle, we calculate the Affine relationship between them and get the inversed Affine Matrix. See function `getinvAffine()`.
5. We then use function `isinTri()` to determine if a point is in new triangle, if yes then applied inverse affine transformation to wrap color. After we got the warped images, we used alpha blending with alpha to get the morphed image.
6. By interpolating coordinates and image values, we got 120 images which represent the intermediate process.
7. We use ffmpeg to create video from images.  

### 2 Video

![Link](https://youtu.be/a_P9SnP5izw)  

### 3 Appendx  


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Oct  3 14:46:46 2017

@author: sunli
"""

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay


def showtri(img,tri,points):
    ## In this function we show the triangles calculated from Delaunay function
    plt.imshow(img)
    plt.triplot(points[:, 0], points[:, 1], tri.simplices.copy())
    plt.plot(points[:, 0], points[:, 1], 'o')
    plt.draw()
    plt.pause(2)

def getTri(img):
    ##We set the image board (0,0), (0,149), (149,0), (149,149)
    board = np.array([[0, 0], [149, 149], [0, 149], [149, 0]], dtype='float32')
    plt.close()
    plt.imshow(img)
    ## Choose 12 points by hand
    print('Please click 12 points (mouth*3) nose(1) eye(2) ear(2) fronhead(1) jaw(1)')
    points = np.array(plt.ginput(12))
    plt.imshow(img)
    points = np.concatenate((board, points))
    ## Calculate triangles
    tri = Delaunay(points)
    showtri(img,tri,points)
    return tri,points


def getinvAffine(points,newpoints):
    
    #In this function we have 
    #points -- the triangle points before Affine and 
    #newpoints -- the triangle points after Affine. 
    #Through this process we can calculate the Affine matrix shown below.
    X = np.array([np.concatenate((points[0],[1,0,0,0])),
                  np.concatenate(([0,0,0],points[0],[1])),
                  np.concatenate((points[1], [1, 0, 0, 0])),
                  np.concatenate(([0, 0, 0], points[1], [1])),
                  np.concatenate((points[2], [1, 0, 0, 0])),
                  np.concatenate(([0, 0, 0], points[2], [1])),
                  ])
    x = np.concatenate((newpoints[0],newpoints[1],newpoints[2]))
    A = np.linalg.solve(X,x)
    A = np.concatenate((A,[0,0,1])).reshape((3,3))
    return np.linalg.inv(A)


def isinTri(tripoints,p):
    ## Barycentric Algorithm to determine if a point is in a triangle given their 
    ## coordinates.
    p1 = tripoints[0]
    p2 = tripoints[1]
    p3 = tripoints[2]
    alpha = ((p2[1] - p3[1]) * (p[0] - p3[0]) + (p3[0] - p2[0]) * (p[1] - p3[1])) /\
    ((p2[1] - p3[1]) * (p1[0] - p3[0]) + (p3[0] - p2[0]) * (p1[1] - p3[1]))
    beta = ((p3[1] - p1[1]) * (p[0] - p3[0]) + (p1[0] - p3[0]) * (p[1] - p3[1])) / \
    ((p2[1] - p3[1]) * (p1[0] - p3[0]) + (p3[0] - p2[0]) * (p1[1] - p3[1]))
    gamma = 1 - alpha - beta
    return alpha>0 and beta>0 and gamma>0

def invwrapcolor(img,point):
    ## Use 2D interpolation to find the color given the orignial image and the points 
    ## coordinate which is not integar
    p0int = int(point[0]//1)
    p0dec = point[0]%1
    p1int = int(point[1]//1)
    p1dec = point[1]%1
    if p1int==149 or p0int==149:
        return img[p1int][p0int]
    c = (1-p0dec)*(1-p1dec)*img[p1int][p0int]+ \
        (1 - p0dec) * p1dec*img[p1int+1][p0int]+ \
        p0dec * (1- p1dec)*img[p1int][p0int+1]+ \
        p0dec * p1dec*img[p1int+1][p0int+1]
    return c


def mergetri(img1,img2, tri1,points1,tri2,points2,alpha=0.5):
    ## Main function, merge two image
    
    ## Create two tmp plot for two image
    newplot1 = np.zeros(img1.shape,dtype='float32')
    newplot2 = np.zeros(img2.shape,dtype='float32')
    
    ## Coordinates
    newpcoord = [[i, j] for i in range(150) for j in range(150)]

    #For each triangle
    for i in range(len(tri1.simplices)):
        
        ## Get triangle coordinates
        t1points = points1[tri1.simplices][i]
        t2points = points2[tri1.simplices][i]

        ## Calculate merged triangle coordinates
        newtri = t1points*alpha + t2points*(1-alpha)


        #Get Affine matrix
        A1 = getinvAffine(t1points, newtri)
        A2 = getinvAffine(t2points, newtri)

        # For each points in new plot, if it is in the new triangle, inverse wrap 
        # its color
        for i in range(len(newpcoord)):
            if isinTri(newtri,newpcoord[i]):
                oldp = A1.dot(np.concatenate((newpcoord[i],[1])))
                newplot1[newpcoord[i][1]][newpcoord[i][0]] = \
                alpha*invwrapcolor(img1,oldp)
                oldp = A2.dot(np.concatenate((newpcoord[i], [1])))
                newplot2[newpcoord[i][1]][newpcoord[i][0]] = \
                (1-alpha) * invwrapcolor(img2, oldp)

    #Merge two plots
    return newplot1+newplot2



if __name__ == '__main__' :
    
    
    #Read Images
    path = './csFaculty'
    alpha = 0.5
    photolist = os.listdir(path)
    os.chdir(path)
    #For each pair of images
    for ii in range(len(photolist)-1):
        filename1 = photolist[ii]
        filename2 = photolist[ii+1]

        # Read images
        img1 = plt.imread(filename1)
        img2 = plt.imread(filename2)

        # Convert int to float data type
        img1 = np.float32(img1)/256
        img2 = np.float32(img2)/256


        #Get triangulation
        tri1, points1 = getTri(img1)
        tri2, points2 = getTri(img2)
        
        #Generate 120 intermidiate plots
        for i in range(121):
            alpha = i/120
            plt.close()
            a = mergetri(img1, img2, tri1, points1, tri2, points2, alpha)
            plt.imshow(a)
            plt.imsave('ret'+str(ii)+str(ii+1)+str(i)+'.png',a)

